In [235]:
from sklearn.datasets import load_digits
%matplotlib inline
import matplotlib.pyplot as plt
digits = load_digits()
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas  as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV , GridSearchCV
import numpy as np
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.decomposition import PCA



In [236]:
df = pd.read_csv("heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [237]:
label_encoder = LabelEncoder()

In [238]:
df["Sex"] = label_encoder.fit_transform(df["Sex"])
df["RestingECG"] = label_encoder.fit_transform(df["RestingECG"])
df["ExerciseAngina"] = label_encoder.fit_transform(df["ExerciseAngina"])
df["ST_Slope"] = label_encoder.fit_transform(df["ST_Slope"])
df["ChestPainType"] = label_encoder.fit_transform(df["ChestPainType"])
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0


In [239]:
def remove_outliers(df, column_names, z_threshold=3):
    for column_name in column_names:
        # Step 1: Calculate Z-score for the specified column
        z_scores = (df[column_name] - df[column_name].mean()) / df[column_name].std()

        # Step 2: Identify outliers based on Z-scores
        outliers = np.abs(z_scores) > z_threshold

        # Step 3: Remove outliers from the DataFrame
        cleaned_df = df[~outliers]

    return cleaned_df

In [240]:
df = remove_outliers(df , ["RestingBP" , "Cholesterol" , "MaxHR"])
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0


In [241]:
scaler = StandardScaler()
columns_to_standardize = ["RestingBP" , "Cholesterol" , "MaxHR"]
df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

In [242]:
all_model_params = {
    "svm": {
        "model": svm.SVC(gamma="auto"),
        "params" :
        {
            
        }
    },
    "random_forest": {
        "model": RandomForestClassifier(),
        "params": {
        }
    },
    "gaussian_nb": {
        "model": GaussianNB(),
        "params": {}
    },
    "multinomial_nb": {
        "model": MultinomialNB(),
        "params": {}
    },
    "logisticregression": {
        "model": LogisticRegression(),
        "params": {}
    },
    "decision_tree": {
        "model": DecisionTreeClassifier(),
        "params": {
        }
    },
    "XG Boost": {
        "model": xgb.XGBClassifier(),
        "params": {
        }
    }
}



In [243]:
y = df["HeartDisease"]
X = df.drop(["HeartDisease"] , axis = 1)
X.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,40,1,1,0.411171,0.824126,0,1,1.385787,0,0.0,2
1,49,0,2,1.491525,-0.174164,0,1,0.754213,0,1.0,1
2,37,1,1,-0.129006,0.769174,0,2,-1.535243,0,0.0,2
3,48,0,0,0.303135,0.137229,0,1,-1.140509,1,1.5,1
4,54,1,2,0.951348,-0.036784,0,1,-0.587882,0,0.0,2


In [244]:
per_components = 0.95  # Choose the number of components to keep
pca = PCA(n_components=per_components)
X = pca.fit_transform(X)
X = pd.DataFrame(X)
X.head()

,0,1
0,13.586368,-0.945714
1,4.502130,-0.583388
2,16.474461,0.659064
3,5.415083,1.728570
4,-0.482338,-1.031707


In [245]:
X_train, X_test, y_train, y_test = train_test_split(digits.data,digits.target, test_size=0.2)

In [246]:
scores = []
for model_name , model_params in all_model_params.items():
    clf = RandomizedSearchCV(model_params["model"] , model_params["params"] , cv=5 ,n_iter=10)
    clf.fit(X_train , y_train)
    scores.append({
        "model" : model_name,
        "best score": clf.best_score_,
        "best params" : clf.best_params_
    })

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\m

In [247]:
df = pd.DataFrame(scores , columns=["model" , "best score" , "best params"])
df

,model,best score,best params
0,svm,0.359096,{}
1,random_forest,0.974959,{}
2,gaussian_nb,0.782905,{}
3,multinomial_nb,0.897004,{}
4,logisticregression,0.963814,{}
5,decision_tree,0.844803,{}
6,XG Boost,0.959640,{}


In [248]:
max_score_index = df["best score"].idxmax()
# Get the row with the highest score
best_model_row = df.loc[max_score_index]
best = pd.DataFrame(best_model_row)
best

,1
model,random_forest
best score,0.974959
best params,{}


In [249]:
clf.score(X_test , y_test)

0.9694444444444444

In [251]:
clf.estimator

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)